# Imports

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
from sympy.solvers.diophantine.diophantine import length

from _config import PATH_RAW_DTU_SOLAR_STATION, PKL_PROCESSED_STEP1_DTU_SOLAR_STATION

# Load DTU Solar Station data
The following code loads all csv files into one pandas dataframe, then it adds the timestamp as the index in pandas, then we ensure that the entire range of observations in a one minute interval is present and if not, then add it to the dataframe as nan values.

Then we show some information about the data.

In [2]:
# Get all CSV files in the path folder
csv_files = list(PATH_RAW_DTU_SOLAR_STATION.glob("*.csv"))

# Read each CSV file and combine them into a single DataFrame
dfs = [pd.read_csv(f) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)
df.set_index('Time(utc)', inplace=True)
df.index = pd.to_datetime(df.index)

# Add missing indexes to the period (columns will be filled with NaN values.)
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq="1min")
df = df.reindex(full_range)
df.sort_index(inplace=True)
df_raw = df.copy()

print(df.shape)
print(df.columns)
display(df.describe().round(3), df.info())
df.head()

(5260805, 12)
Index(['GHI', 'DNI', 'DHI', 'LWD', 'wind_speed_avg', 'wind_dir_avg',
       'air_temperature', 'air_pressure', 'relative_humidity',
       'rain_accumulation', 'rain_duration', 'rain_intensity'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5260805 entries, 2015-01-01 00:00:00 to 2025-01-01 08:04:00
Freq: min
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   GHI                float64
 1   DNI                float64
 2   DHI                float64
 3   LWD                float64
 4   wind_speed_avg     float64
 5   wind_dir_avg       float64
 6   air_temperature    float64
 7   air_pressure       float64
 8   relative_humidity  float64
 9   rain_accumulation  float64
 10  rain_duration      float64
 11  rain_intensity     float64
dtypes: float64(12)
memory usage: 521.8 MB


,GHI,DNI,DHI,LWD,wind_speed_avg,wind_dir_avg,air_temperature,air_pressure,relative_humidity,rain_accumulation,rain_duration,rain_intensity
count,5031319.000,4965446.000,4993921.000,3073445.000,5086375.000,5086375.000,5086013.000,5086203.000,5086203.000,4963979.000,4963979.000,4963979.000
mean,118.681,122.276,57.042,753.191,2.732,204.257,9.897,1007.485,73.333,0.001,1.953,0.065
std,207.283,261.393,92.284,9781.588,1.774,89.644,6.587,10.670,13.849,0.014,9.794,0.857
min,-9.758,-12.520,-23.610,-7.325,-0.980,-0.139,-10.100,958.200,12.200,-0.919,-0.946,-0.909
25%,-1.149,-0.303,-1.079,293.227,1.400,123.000,4.700,1001.000,64.900,0.000,0.000,0.000
50%,3.305,-0.006,3.613,324.874,2.400,231.000,9.500,1008.000,76.800,0.000,0.000,0.000
75%,149.400,8.135,83.760,349.682,3.700,274.000,15.000,1014.000,84.400,0.000,0.000,0.000
max,1690.000,1001.190,882.000,311300.000,26.700,359.000,31.600,1044.000,94.400,2.080,110.000,123.400


None

,GHI,DNI,DHI,LWD,wind_speed_avg,wind_dir_avg,air_temperature,air_pressure,relative_humidity,rain_accumulation,rain_duration,rain_intensity
2015-01-01 00:00:00,0.000325,NaN,-0.000002,NaN,5.0,278.0,4.8,1020.0,81.5,0.0,0.0,0.0
2015-01-01 00:01:00,NaN,NaN,NaN,NaN,4.7,264.0,4.8,1020.0,81.6,0.0,0.0,0.0
2015-01-01 00:02:00,NaN,NaN,NaN,NaN,3.3,269.0,4.8,1020.0,81.6,0.0,0.0,0.0
2015-01-01 00:03:00,NaN,NaN,NaN,NaN,3.6,247.0,4.8,1020.0,81.7,0.0,0.0,0.0
2015-01-01 00:04:00,NaN,NaN,NaN,NaN,2.2,228.0,4.8,1020.0,81.7,0.0,0.0,0.0


# Preprocess DTU Solar Station Data

## Remove features
Drop column ``LWD`` - a lot of missing data. \
Drop column ``rain_accumulation`` - based on manual reset time of hardware and thus not meaningful. \
Drop column ``GHI`` - This is a sum of DNI and DHI.

In [3]:
df = df_raw.copy()
# We observed alot of missing values in LWD
df.drop(columns=['LWD'], inplace=True)
df.drop(columns=['rain_accumulation'], inplace=True)
df.drop(columns=['GHI'], inplace=True)

## Set bad periods to NaN
Certain periods in the time series data has bad data and can be seen in the previous time series plot. \
I set all feature to NaN for the periods to eliminate the use of them in the dataset - but keep the time index.

In [4]:
import numpy as np

df = df.copy()
# display(df.loc['2021-01-05'].head())
mask = (df.index > "2021-01-04") & (df.index < "2021-02-23")
df.loc[mask] = np.nan
# display(df.loc['2021-01-05'].head())

# display(df.loc['2015-03-11'].head())
mask = df.index < "2015-03-12"
df.loc[mask] = np.nan
# display(df.loc['2015-03-11'].head())

# display(df.loc['2018-11-05'].head())
mask = (df.index < "2018-11-07") & (df.index > "2018-08-12")
df.loc[mask] = np.nan
# display(df.loc['2018-11-05'].head())

## Imputation/Interpolation
Instead of discarding samples and thus potential data for training, then missing values for up to 15 points are imputed/interpolated using varying techniques in order to fill the data.\
If the missing values are greater than 15, then we discard the period.

In [5]:
from collections import Counter

df_imputed = df_raw.copy()

# Create a boolean mask for rows with any NaN value.
mask = df_imputed.isna().any(axis=1).values  # using .values for easy iteration

# Identify contiguous gap groups (list of lists of integer positions)
gap_groups = []
current_group = []
for i, is_nan in enumerate(mask):
    if is_nan:
        current_group.append(i)
    else:
        if current_group:
            gap_groups.append(current_group)
            current_group = []
if current_group:
    gap_groups.append(current_group)
        
length_counts = Counter(len(lst) for lst in gap_groups)
occurence_df = pd.DataFrame.from_dict(length_counts, orient='index').reset_index()
occurence_df.columns = ['Missing Sequence Length', 'Number of Occurences']
occurence_df.sort_values(by='Missing Sequence Length', inplace=True)
print(occurence_df.to_latex(index=False))
occurence_df

\begin{tabular}{rr}
\toprule
Missing Sequence Length & Number of Occurences \\
\midrule
1 & 23 \\
2 & 88 \\
3 & 35 \\
4 & 6 \\
5 & 5 \\
6 & 6 \\
7 & 1 \\
8 & 2 \\
12 & 2 \\
23 & 1 \\
26 & 1 \\
31 & 1 \\
34 & 1 \\
36 & 1 \\
53 & 1 \\
54 & 1 \\
63 & 1 \\
101 & 1 \\
121 & 1 \\
126 & 1 \\
205 & 1 \\
279 & 1 \\
485 & 1 \\
1146 & 1 \\
4384 & 1 \\
5630 & 1 \\
5651 & 1 \\
7070 & 1 \\
12738 & 1 \\
70898 & 1 \\
77478 & 1 \\
2036160 & 1 \\
\bottomrule
\end{tabular}



,Missing Sequence Length,Number of Occurences
2,1,23
3,2,88
4,3,35
10,4,6
7,5,5
28,6,6
29,7,1
26,8,2
30,12,2
31,23,1


In [6]:
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline


def impute_nan_gaps(df):
    """
    Impute gaps (rows with any NaN values) in a timeseries DataFrame
    using different interpolation methods based on the gap length.

    For each gap that is "sandwiched" by valid (non-NaN) rows:
      - Gap length == 1: fill with the average of the previous and next valid rows.
      - Gap length 2 to 5: use linear interpolation.
      - Gap length 6 to 10: use polynomial interpolation (order 2).
          (If sufficient surrounding points are not available, fallback to linear.)
      - Gap length 11 to 15: use spline interpolation (order 2).
          (Again, if not enough valid surrounding points are available, fallback to linear.)
      - Gaps >15: leave the gap as-is.

    Parameters:
      df : pandas DataFrame with a timeseries index.

    Returns:
      df_imputed : a new DataFrame with the imputed values.
    """
    # Work on a copy to avoid modifying the original DataFrame.
    df_imputed = df.copy()

    # Create a boolean mask for rows with any NaN value.
    mask = df_imputed.isna().any(axis=1).values  # using .values for easy iteration

    # Identify contiguous gap groups (list of lists of integer positions)
    gap_groups = []
    current_group = []
    for i, is_nan in enumerate(mask):
        if is_nan:
            current_group.append(i)
        else:
            if current_group:
                gap_groups.append(current_group)
                current_group = []
    if current_group:
        gap_groups.append(current_group)

    # Process each gap group
    for group in gap_groups:
        gap_length = len(group)
        start = group[0]
        end = group[-1]
        # Ensure the gap is sandwiched by valid rows
        if start == 0 or end == len(df_imputed) - 1:
            continue
        # Skip gaps longer than 15 rows
        if gap_length > 15:
            df_imputed.iloc[start:end + 1] = np.nan
            continue

        # Decide which interpolation method to use
        if gap_length == 1:
            method = 'average'
        elif 2 <= gap_length <= 5:
            method = 'linear'
        elif 6 <= gap_length <= 10:
            method = 'polynomial'
        elif 11 <= gap_length <= 15:
            method = 'spline'

        # Process each column separately.
        for col in df_imputed.columns:
            # Check if any row in this gap is missing for the column.
            # (It might be that only some columns are missing in the gap.)
            gap_vals = df_imputed.iloc[start:end + 1][col]
            if not gap_vals.isna().any():
                continue  # nothing to fill for this column in this gap

            # Get the endpoint values (assumed valid) for this column.
            prev_val = df_imputed.iat[start - 1, df_imputed.columns.get_loc(col)]
            next_val = df_imputed.iat[end + 1, df_imputed.columns.get_loc(col)]
            if pd.isna(prev_val) or pd.isna(next_val):
                # Safety check: if either endpoint is missing, skip imputation for this gap/column.
                continue

            if method == 'average':
                # Gap length 1: fill with the average of the two endpoints.
                fill_val = (prev_val + next_val) / 2
                df_imputed.iat[start, df_imputed.columns.get_loc(col)] = fill_val

            elif method == 'linear':
                # Linear interpolation over the gap.
                # For each missing row j (0-indexed in the gap), compute:
                # interpolated_val = prev_val + (next_val - prev_val) * (j+1)/(gap_length+1)
                for j in range(gap_length):
                    interpolated_val = prev_val + (next_val - prev_val) * (j + 1) / (gap_length + 1)
                    df_imputed.iat[start + j, df_imputed.columns.get_loc(col)] = interpolated_val

            elif method == 'polynomial':
                # Attempt quadratic interpolation (order=2).
                # We need at least 3 points. We try to use:
                #   - the row two steps before the gap (if available)
                #   - the row immediately before the gap
                #   - the row immediately after the gap
                indices = []
                values = []
                if start - 2 >= 0:
                    indices.append(start - 2)
                    values.append(df_imputed.iat[start - 2, df_imputed.columns.get_loc(col)])
                # Always include the row immediately before the gap.
                indices.append(start - 1)
                values.append(prev_val)
                # Include the row immediately after the gap.
                indices.append(end + 1)
                values.append(next_val)

                # If any of these are missing, fallback to linear interpolation.
                if any(pd.isna(v) for v in values) or len(indices) < 3:
                    for j in range(gap_length):
                        interpolated_val = prev_val + (next_val - prev_val) * (j + 1) / (gap_length + 1)
                        df_imputed.iat[start + j, df_imputed.columns.get_loc(col)] = interpolated_val
                else:
                    # Fit a quadratic polynomial.
                    x = np.array(indices)
                    y = np.array(values)
                    coeffs = np.polyfit(x, y, 2)
                    poly = np.poly1d(coeffs)
                    # Fill in the gap rows by evaluating the polynomial at the corresponding positions.
                    for idx in range(start, end + 1):
                        df_imputed.iat[idx, df_imputed.columns.get_loc(col)] = poly(idx)

            elif method == 'spline':
                # Spline interpolation (order=2). We try to use two valid points before and after if possible.
                indices = []
                values = []
                if start - 2 >= 0:
                    indices.append(start - 2)
                    values.append(df_imputed.iat[start - 2, df_imputed.columns.get_loc(col)])
                if start - 1 >= 0:
                    indices.append(start - 1)
                    values.append(prev_val)
                if end + 1 < len(df_imputed):
                    indices.append(end + 1)
                    values.append(next_val)
                if end + 2 < len(df_imputed):
                    indices.append(end + 2)
                    values.append(df_imputed.iat[end + 2, df_imputed.columns.get_loc(col)])

                # We need at least 3 points for spline interpolation.
                if len(indices) < 3 or any(pd.isna(v) for v in values):
                    # Fallback to linear if not enough valid points.
                    for j in range(gap_length):
                        interpolated_val = prev_val + (next_val - prev_val) * (j + 1) / (gap_length + 1)
                        df_imputed.iat[start + j, df_imputed.columns.get_loc(col)] = interpolated_val
                else:
                    # Create a spline (k=2 for quadratic spline, s=0 forces interpolation).
                    x = np.array(indices)
                    y = np.array(values)
                    spline = UnivariateSpline(x, y, k=2, s=0)
                    for idx in range(start, end + 1):
                        df_imputed.iat[idx, df_imputed.columns.get_loc(col)] = spline(idx)

    return df_imputed


# Example usage:
# Suppose `df` is your timeseries DataFrame.
df: pd.DataFrame = impute_nan_gaps(df)
# Now `imputed_df` has gaps imputed for gap sizes 1-15 using different techniques.


## Clip invalid values
Solar radiation is not negative. Thus ``DNI`` and ``DHI`` will be clipped to zero when values are negative.

In [7]:
# Values below 0 are set to 0 for GHI, DNI, and DHI
df[['DNI', 'DHI']] = df[['DNI', 'DHI']].clip(lower=0)

## Solar Altitude
The solar altitude is the angle of the sun above the horizon. \

In [8]:
from astral import LocationInfo, Observer
from astral.sun import elevation

# Define location information for DTU Lyngby (Building 119)
location = LocationInfo(
    name="DTU Lyngby (Building 119)",
    region="Denmark",
    latitude=55.79064,
    longitude=12.52505,
)
observer = Observer(location.latitude, location.longitude, 50)  # 50 meters above sea level


# Define a function to compute solar altitude at a given datetime
def compute_solar_altitude(dt):
    return elevation(observer, dt)


# Apply the function to each datetime in the index and add a new column
df['solar_altitude'] = df.index.to_series().apply(compute_solar_altitude)

In [9]:
from pvlib.location import Location
location = Location(
    latitude=55.79064,
    longitude=12.52505,
    altitude=50,  # in meters
    name="DTU Lyngby (Building 119)",
    tz='Europe/Copenhagen'  # Adjust timezone appropriately
)
# Compute clear-sky Direct Normal Irradiance (DNI)
clearsky = location.get_clearsky(times=df.index, model='ineichen')

# Add DNI to your dataframe
df['DNI_CLEAR_SKY'] = clearsky['dni']
df['DHI_CLEAR_SKY'] = clearsky['dhi']
# df

## Set entire row to NaN
To visualize and clearly see which datasamples are valid, then any row after preprocessing with a missing value will have all their feature set to NaN.

In [10]:
df.loc[df.isnull().any(axis=1)] = np.nan

## Set entire row to NaN if solar altitude is below 0

In [11]:
# mask = df['solar_altitude'] < 0
# df.loc[mask, :] = np.nan

## Drop NaN rows

In [12]:
# df.dropna(inplace=True)

## Save preprocessing for later loading.

In [13]:
df.to_pickle(PKL_PROCESSED_STEP1_DTU_SOLAR_STATION)

In [14]:
df

,DNI,DHI,wind_speed_avg,wind_dir_avg,air_temperature,air_pressure,relative_humidity,rain_duration,rain_intensity,solar_altitude,DNI_CLEAR_SKY,DHI_CLEAR_SKY
2015-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 08:00:00,0.0,0.0,5.2,235.0,8.3,990.0,85.5,50.0,0.3,1.553152,128.279617,1.768857
2025-01-01 08:01:00,0.0,0.0,9.3,227.0,8.3,989.8,85.6,60.0,0.4,1.641343,135.825312,1.925021
2025-01-01 08:02:00,0.0,0.0,2.1,280.0,8.3,990.0,85.5,40.0,0.5,1.729506,143.463186,2.085210
2025-01-01 08:03:00,0.0,0.0,7.6,260.0,8.3,989.9,85.6,50.0,0.5,1.817622,151.178202,2.249067
